In [1]:
import requests
import sys
import pandas as pd

import wikidataintegrator as wdi




query = '''
SELECT DISTINCT * WHERE {
    {?cmpnd wdt:P279 wd:Q11173 .} UNION
    {?cmpnd wdt:P31 wd:Q11173 .} UNION
    {?cmpnd wdt:P31 wd:Q12140 .} UNION
    {?cmpnd wdt:P31 wd:Q12140 .} UNION
    {?cmpnd wdt:P662 [] .} UNION
    {?cmpnd wdt:P231 [] .} UNION
    {?cmpnd wdt:P235 [] .} UNION
    {?cmpnd wdt:P234 [] .}
    
    %s

}

'''


In [2]:
chem_data_dict = {}

prop_map = [
    ('', 'cmpnd'), 
    ('', 'article'), 
    ('P2067', 'mass'), 
    ('P661', 'chemSpider'), 
    ('P662', 'pubchem_cid'), 
    ('P652', 'unii'), 
    ('P486', 'mesh_id'),
    ('P665', 'kegg_id'), 
    ('P672', 'mesh_code'), 
    ('P683', 'chebi'),
    ('P715', 'drugbank'),
    ('P592', 'chembl'),
    ('P595', 'iuphar'),
    ('P231', 'cas'),
    ('P233', 'csmiles'), 
    ('P234', 'inchi'), 
    ('P235', 'inchi_key'), 
    ('P274', 'chemical_formula'), 
    ('P267', 'atc_code'), 
    ('P2017', 'ismiles')
]

columns = [x[1] for x in prop_map]

article_query = '''
    OPTIONAL {
        ?article schema:about ?cmpnd .
        ?article schema:inLanguage "en" .
        FILTER (SUBSTR(str(?article), 1, 25) = "https://en.wikipedia.org/") .
    }
'''

prop_query = '''
OPTIONAL{{?cmpnd wdt:{} ?{}.}}
'''

base_query = query % (article_query)
print(base_query)
wd_chem_data = pd.DataFrame(columns=columns)

results = wdi.wdi_core.WDItemEngine.execute_sparql_query(base_query)

print(len(results['results']['bindings']))

count = 0
for x in results['results']['bindings']:
    qid = x['cmpnd']['value']
    wd_chem_data.loc[count, 'cmpnd'] = qid
    if 'article' in x:
        wd_chem_data.loc[count, 'article'] = x['article']['value']
    
    count += 1
    
    if count % 5000 == 0:
        print(count)
        #print(wd_chem_data.head())
    
wd_chem_data.head()
# cc = 0
# while True:
# #     print(query % (100000 * cc))
#     results = wdi.wdi_core.WDItemEngine.execute_sparql_query(query % (5000 * cc))
#     cc += 1
    
#     if len(results['results']['bindings']) == 0:
#         break
        
#     if len(columns) > 0:
#         columns.extend(results['head']['vars'])
        
#     for x in results['results']['bindings']:
#         qid = x['cmpnd']['value']
#         if qid not in chem_data_dict:
#             chem_data_dict[qid] = {}
            
#         for key, val in x.items():
#             if key == 'cmpnd':
#                 continue
                
#             if key not in chem_data_dict[qid]:
#                 chem_data_dict[qid][key] = set()
                
#             chem_data_dict[qid][key].add(val['value'])
            
        
        
#     print(cc)
            


SELECT DISTINCT * WHERE {
    {?cmpnd wdt:P279 wd:Q11173 .} UNION
    {?cmpnd wdt:P31 wd:Q11173 .} UNION
    {?cmpnd wdt:P662 [] .} UNION
    {?cmpnd wdt:P231 [] .} UNION
    {?cmpnd wdt:P235 [] .} UNION
    {?cmpnd wdt:P234 [] .}
    
    
    OPTIONAL {
        ?article schema:about ?cmpnd .
        ?article schema:inLanguage "en" .
        FILTER (SUBSTR(str(?article), 1, 25) = "https://en.wikipedia.org/") .
    }


}


158964
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000


,cmpnd,article,mass,chemSpider,pubchem_cid,unii,mesh_id,kegg_id,mesh_code,chebi,drugbank,chembl,iuphar,cas,csmiles,inchi,inchi_key,chemical_formula,atc_code,ismiles
0,http://www.wikidata.org/entity/Q161267,https://en.wikipedia.org/wiki/1%2C1%2C1-Triflu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://www.wikidata.org/entity/Q161264,https://en.wikipedia.org/wiki/1%2C1%2C1%2C2-Te...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://www.wikidata.org/entity/Q161275,https://en.wikipedia.org/wiki/1%2C1%2C2%2C2-Te...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q161276,https://en.wikipedia.org/wiki/1%2C1%2C2-Trichl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://www.wikidata.org/entity/Q161271,https://en.wikipedia.org/wiki/Tetrabromoethane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
wd_chem_data.to_csv('base_wd_chem_ids.csv')

In [51]:
wd_chem_data = pd.read_csv('base_wd_chem_ids.csv', low_memory=False)

In [52]:
wd_chem_data.duplicated(subset='cmpnd').any()
wd_chem_data.head()

,Unnamed: 0,cmpnd,article,mass,chemSpider,pubchem_cid,unii,mesh_id,kegg_id,mesh_code,...,drugbank,chembl,iuphar,cas,csmiles,inchi,inchi_key,chemical_formula,atc_code,ismiles
0,0,http://www.wikidata.org/entity/Q161267,https://en.wikipedia.org/wiki/1%2C1%2C1-Triflu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,http://www.wikidata.org/entity/Q161264,https://en.wikipedia.org/wiki/1%2C1%2C1%2C2-Te...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,http://www.wikidata.org/entity/Q161275,https://en.wikipedia.org/wiki/1%2C1%2C2%2C2-Te...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,http://www.wikidata.org/entity/Q161276,https://en.wikipedia.org/wiki/1%2C1%2C2-Trichl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,http://www.wikidata.org/entity/Q161271,https://en.wikipedia.org/wiki/Tetrabromoethane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
cdf_ni = wd_chem_data.set_index(wd_chem_data['cmpnd'])
cdf_ni.head()


,Unnamed: 0,cmpnd,article,mass,chemSpider,pubchem_cid,unii,mesh_id,kegg_id,mesh_code,...,drugbank,chembl,iuphar,cas,csmiles,inchi,inchi_key,chemical_formula,atc_code,ismiles
cmpnd,,,,,,,,,,,,,,,,,,,,,
http://www.wikidata.org/entity/Q161267,0,http://www.wikidata.org/entity/Q161267,https://en.wikipedia.org/wiki/1%2C1%2C1-Triflu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://www.wikidata.org/entity/Q161264,1,http://www.wikidata.org/entity/Q161264,https://en.wikipedia.org/wiki/1%2C1%2C1%2C2-Te...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://www.wikidata.org/entity/Q161275,2,http://www.wikidata.org/entity/Q161275,https://en.wikipedia.org/wiki/1%2C1%2C2%2C2-Te...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://www.wikidata.org/entity/Q161276,3,http://www.wikidata.org/entity/Q161276,https://en.wikipedia.org/wiki/1%2C1%2C2-Trichl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://www.wikidata.org/entity/Q161271,4,http://www.wikidata.org/entity/Q161271,https://en.wikipedia.org/wiki/Tetrabromoethane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
prop_query = '''
OPTIONAL{{?cmpnd wdt:{} ?{}.}}
'''

for x in prop_map[2:]:
    prop, label = x
    
    q = prop_query.format(prop, label)
    fq = query % (q)
#     print(fq)
    r = wdi.wdi_core.WDItemEngine.execute_sparql_query(fq)
    
    for count, g in enumerate(r['results']['bindings']):
        qid = g['cmpnd']['value']
        
        if label in g:
#             tvec = wd_chem_data['cmpnd'].isin([qid])
#             print(tvec)
#             print(len(tvec))
#             print(tvec.any())
#             print(tvec.describe())
#             print(tvec.value_counts())
#             pd_value = wd_chem_data.loc[tvec, 'cmpnd']
#             print(pd_value)
            try:
                if pd.isnull(cdf_ni.loc[qid, label]):
                    cdf_ni.loc[qid, label] = g[label]['value']
                else:
                    stuff = set(cdf_ni.loc[qid, label].split('|'))
                    
                    stuff.add(g[label]['value'])
                    cdf_ni.loc[qid, label] = '|'.join(stuff)
            except KeyError:
                pass
            

        count += 1

        if count % 5000 == 0:
            print(count)
#             print(cdf_ni.head())

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
5000
100

In [64]:
cdf_ni.head()
cdf_ni.to_csv('wd_chem_2017-01-27.csv')

In [65]:
cdf_ni.count()

Unnamed: 0          158964
cmpnd               158964
article              16691
mass                147255
chemSpider          122616
pubchem_cid         144803
unii                 58286
mesh_id                947
kegg_id              12957
mesh_code                3
chebi                82489
drugbank              7167
chembl               49596
iuphar                5889
cas                  71443
csmiles             151661
inchi               148825
inchi_key           151447
chemical_formula    148138
atc_code              1725
ismiles              84233
dtype: int64

In [28]:
import pprint
import urllib
import requests
import pandas as pd

chem_parameters = [
    'title',
    'SystematicName',
    'OtherNames',
    'synonyms',
    'IUPHAR_ligand',
    'CASNo',
    'CAS_number',
    'UNII',
    'SMILES',
    'KEGG',
    'ChEMBL',
    'ChemSpiderID',
    'DrugBank',
    'PubChem',
    'StdInChI',
    'StdInChIKey',
    'InChI',
    'InChIKey',
    'ATCCode_prefix',
    'ATC_prefix',
    'ATCCode_suffix',
    'ATC_suffix',
    'ATC_Supplemental',
    'molecular_weight',
    'page_title'
]

s = requests.Session()

def parse_chembox(pages):
    # querying one page at a time is most conventient to code

    url = 'https://en.wikipedia.org/w/api.php'

    if not pages:
        pages = ['Imatinib', 'Vemurafenib']

    params = {
            'action': 'query',
            'titles': '|'.join(pages),
            'prop': 'revisions',
            'rvprop': 'content',
            'format': 'json'
        }
    r = requests.get(url=url, params=params).json()
    #pprint.pprint(r)
    for zz in [x for x in r['query']['pages'].values()]:
        current_page_title = zz['title']
        page_text = zz['revisions'][0]['*']
        #pprint.pprint(page_text)
        #print(page_text)

        for line in page_text.split('\n'):
            value = ''
            chem_param = ''
            if line.strip().startswith('|'):
                splits = line[1:].strip().split('=', maxsplit=1)
                if splits[-1].strip():
                    value = splits[-1].strip()
                chem_param = splits[0].strip()
                
#                 if splits[-1].strip() == '=':
#                     chem_param = chem_param[:-1]

#                 value = line.split('=', maxsplit=1)[-1].strip()

                if chem_param in chem_parameters:
#                     print(chem_param, value)
                    yield {chem_param: value}
#             elif line.strip().startswith("'''"):
#                 name = line.split("'''")[1]
#                 yield {'title': name}

        yield {'page_title': current_page_title}
        


def infobox_backlinks(ib):
    base_url = 'https://en.wikipedia.org/w/api.php'
    for i in ib:

        params = {
            'action': 'query',
            'list': 'embeddedin',
            'eititle': i,
            'eilimit': 500,
            'format': 'json',
            'eifilterredir': 'nonredirects',
            'einamespace': 0
        }

        r = s.get(url=base_url, params=params)
        backlinks = r.json()['query']['embeddedin']

        if 'continue' in r.json():
            cont = r.json()['continue']
            params.update(cont)
            # pprint.pprint(params)
            while True:
                r = s.get(url=base_url, params=params)
                if 'embeddedin' in r.json()['query']:
                    backlinks.extend(r.json()['query']['embeddedin'])

                if 'continue' in r.json():
                    cont = r.json()['continue']
                    params.update(cont)
                else:
                    break

        for z in backlinks:
            yield z


def handle_atc(drug_data):
    prefixes = []
    suffixes = []
    for x in drug_data:
        if 'ATC_prefix' in x.keys() or 'ATCCode_prefix' in x.keys():
            value = list(x.values())[0]
            if not value or value == 'none':
                continue
            prefixes.append(value)
            suffixes.append('')
        elif 'ATC_suffix' in x.keys() or 'ATCCode_suffix' in x.keys():
            try:
                suffixes[-1] = list(x.values())[0]
            except IndexError:
                pass
            
    return [x + suffixes[c] for c, x in enumerate(prefixes)]


infoboxes = ['Template:Infobox_drug', 'Template:Chembox']

columns = [
    'page_title',
    'pageid',
    'SystematicName',
    'OtherNames',
    'synonyms',
    'IUPHAR_ligand',
    'CASNo',
    'CAS_number',
    'UNII',
    'SMILES',
    'KEGG',
    'ChEMBL',
    'ChEBI',
    'ChemSpiderID',
    'DrugBank',
    'PubChem',
    'StdInChI',
    'StdInChIKey',
    'InChI',
    'InChIKey',
    'ATC_code',
    'molecular_weight',
]


        

In [25]:
for x in parse_chembox(["N'-Formylkynurenine", 'Etanautine']):
    print(x) #'Etanautine', 

{'OtherNames': ''}
{'CASNo': '1022-31-7'}
{'PubChem': '910'}
{'SMILES': ''}
{'page_title': "N'-Formylkynurenine"}
{'CAS_number': '857401-00-4'}
{'ATC_prefix': 'N04'}
{'ATC_suffix': 'AB01'}
{'PubChem': ''}
{'DrugBank': ''}
{'ChemSpiderID': '8430627'}
{'molecular_weight': '493.555 g/mol'}
{'StdInChI': '1S/C17H21NO.C9H10N4O4/c1-18(2)13-14-19-17(15-9-5-3-6-10-15)16-11-7-4-8-12-16;1-11-7-6(8(16)12(2)9(11)17)13(4-10-7)3-5(14)15/h3-12,17H,13-14H2,1-2H3;4H,3H2,1-2H3,(H,14,15)'}
{'StdInChIKey': 'UFKLOBYBVCBTBP-UHFFFAOYSA-N'}
{'page_title': 'Etanautine'}


In [29]:
wp_chem_data = pd.DataFrame(columns=columns)
columns.remove('page_title')
columns.remove('ATC_code')
columns.remove('pageid')

for count, art in enumerate(infobox_backlinks(infoboxes)):
    title = urllib.parse.unquote(art['title'])
    all_data = list(parse_chembox([title]))
    
    wp_chem_data.loc[count, 'page_title'] = title
    wp_chem_data.loc[count, 'pageid'] = art['pageid']
    atc_codes = '|'.join(handle_atc(all_data))
#     print(atc_codes)
    wp_chem_data.loc[count, 'ATC_code'] = atc_codes
    
#     print(count, title)
#     pprint.pprint(list(parse_chembox([page_title])))
    
    
    for col in columns:
        wp_chem_data.loc[count, col] = '|'.join([x[col] for x in all_data if col in x.keys()])
        
#     if count > 300:
#         break
    if count % 100 == 0:
        print(count, title)

0 Aspirin
100 Streptomycin
200 Omeprazole
300 Carisoprodol
400 Bismuth subsalicylate
500 Baclofen
600 Obidoxime
700 5-MeO-DET
800 Methoxsalen
900 Chlorprothixene
1000 Amineptine
1100 Triptorelin
1200 Penbutolol
1300 Carglumic acid
1400 Homatropine
1500 Efavirenz/emtricitabine/tenofovir
1600 Sultiame
1700 Doripenem
1800 Pipamperone
1900 Bertilimumab
2000 Libivirumab
2100 Technetium (99mTc) medronic acid
2200 Methylnaltrexone
2300 Propentofylline
2400 Moxaverine
2500 Enclomifene
2600 Amlexanox
2700 Etifelmine
2800 Feprosidnine
2900 Cilomilast
3000 Centalun
3100 L-759,656
3200 Manidipine
3300 Oxypertine
3400 Falecalcitriol
3500 Carbaldrate
3600 Obinutuzumab
3700 Conorfone
3800 Neratinib
3900 Tetraphenylporphine sulfonate
4000 S-23 (drug)
4100 Lubazodone
4200 LY-307,452
4300 Ethinylestradiol/norethisterone
4400 Profadol
4500 Ancarolol
4600 6-APDB
4700 Trazium
4800 Mezepine
4900 Quinaprilat
5000 Nintedanib
5100 Sirukumab
5200 Difebarbamate
5300 Ocaratuzumab
5400 JWH-057
5500 Glucarpidase
56

In [30]:
wp_chem_data

,page_title,pageid,SystematicName,OtherNames,synonyms,IUPHAR_ligand,CASNo,CAS_number,UNII,SMILES,...,ChEMBL,ChemSpiderID,DrugBank,PubChem,StdInChI,StdInChIKey,InChI,InChIKey,ATC_code,molecular_weight
0,Aspirin,1525,,,2-acetoxybenzoic acid<br />acetylsalicylate<br...,4139,,50-78-2,R16CO5Y76E,,...,25,2157,DB00945,2244,1S/C9H8O4/c1-6(10)13-8-5-3-2-4-7(8)9(11)12/h2-...,BSYNRYMUTXBXSQ-UHFFFAOYSA-N,,,A01AD05,180.157 g/mol
1,Ampicillin,1912,,,,,,69-53-4,7C782967RD,O=C(O)[C@@H]2N3C(=O)[C@@H](NC(=O)[C@@H](c1cccc...,...,174,6013,DB00415,6249,1S/C16H19N3O4S/c1-16(2)11(15(22)23)19-13(21)10...,AVKUERGKIZMTKX-NJBDSQKTSA-N,,,J01CA01,349.41 g/mol
2,Amphetamine,2504,,,α-methylphenethylamine,4804,,300-62-9,CK833KGX7E,,...,405,13852819,DB00182,3007,"1S/C9H13N/c1-8(10)7-9-5-3-2-4-6-9/h2-6,8H,7,10...",KWTSXDURSIMDCE-UHFFFAOYSA-N,,,N06BA01,"135.20622&nbsp;g/mol<ref name=""PubChem Header"" />"
3,Amoxicillin,2885,,,amoxicillin ([[international nonproprietary na...,,,26787-78-0,9EM05410Q9,O=C(O)[C@@H]2N3C(=O)[C@@H](NC(=O)[C@@H](c1ccc(...,...,1082,31006,DB01060,33613,1S/C16H19N3O5S/c1-16(2)11(15(23)24)19-13(22)10...,LSQZJLSUYDQPKJ-NJBDSQKTSA-N,,,J01CA04,365.4 g/mol
4,BCG vaccine,4191,,,,,,,,,...,,none,,,,,,,J07AN01,
5,Chloramphenicol,6346,,,,,,56-75-7,66974FR9Q1,,...,130,5744,DB00446,298,1S/C11H12Cl2N2O5/c12-10(13)11(18)14-8(5-16)9(1...,WIIZWVCIJKGZOK-RKDXNWHRSA-N,,,D06AX02,323.1320 g/mol
6,Ciprofloxacin,6773,,,,,,85721-33-1,5E8K9I0O4U,,...,8,2662,DB00537,2764,1S/C17H18FN3O3/c18-13-7-11-14(8-15(13)20-5-3-1...,MYSWGUAQZAJSOK-UHFFFAOYSA-N,,,J01MA02,331.346 g/mol
7,Caffeine,6868,,,Guaranine<!--Pubchem--><br />Methyltheobromine...,407,,58-08-2,3G6A5W338E,,...,113,2424,DB00201,2519,1S/C8H10N4O2/c1-10-4-9-6-5(10)7(13)12(3)8(14)1...,RYYVLZVUVIJVGH-UHFFFAOYSA-N,,,N06BC01,194.19&nbsp;g/mol
8,Carbamazepine,6949,,,,5339,,298-46-4,33CM23913M,,...,108,2457,DB00564,2554,1S/C15H12N2O/c16-15(18)17-13-7-3-1-5-11(13)9-1...,FFGPTBGBLSHEPO-UHFFFAOYSA-N}},,,N03AF01,236.269&nbsp;g/mol
9,Coal tar,7466,,,,,,8007-45-2,,,...,,,,,,,,,D05AA,


In [31]:
wp_chem_data.to_csv('wp_chem_data_2017-01-27.csv')